In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud

def process_msg(text):
    text = re.sub(r'(@|https?)\S+|#', '', text)
    filters = '@!"\'#$%&()*+,-/:;<=>[\\]^_`{|}~\t\n'  # '?' ve '.' kaldırıldı
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = text.lower()
    text = text.strip()

    # Stop words'u kaldırma
    stop_words = set(stopwords.words('turkish'))
    words = word_tokenize(text)
    words_filtered = [word for word in words if word.isalpha() and word not in stop_words]
    text = ' '.join(words_filtered)
    
    return text

# Veriyi yükle
xt = pd.read_excel(r'C:\Users\LENOVO\Desktop\TeknoFest\half29072024_teknofest.xlsx')
xt = xt.dropna()
xt['mesaj'] = xt['mesaj'].astype(str).apply(process_msg)
xt['label'] = xt['label']

# Örnek bir veri kontrolü
print(xt.head())

# Label dağılımını görselleştir
plt.figure(figsize=(10, 6))
xt['label'].value_counts().plot(kind='bar')
plt.title('Label Dağılımı')
plt.xlabel('Label')
plt.ylabel('Frekans')
plt.show()

# "1" labeline sahip mesajların olup olmadığını kontrol et
print(xt['label'].value_counts())

# "1" labeline sahip mesajların kelime bulutu
one_messages = xt[xt['label'] == '1']['mesaj']
if not one_messages.empty:
    text = ' '.join(one_messages)
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('"1" Labeline Sahip Mesajların Kelime Bulutu')
    plt.show()
else:
    print('"1" labeline sahip mesaj bulunamadı.')

# Özellik çıkarımı için TF-IDF vektörizer kullan
vectorizer = TfidfVectorizer()

# Veriyi ve etiketleri ayır
X = vectorizer.fit_transform(xt['mesaj'])
y = xt['label']

# Eğitim ve test verisine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Modelleri tanımla
models = {
    "SVM": SVC(),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Logistic Regression": LogisticRegression()
}

# Modelleri eğit ve sonuçları al
for model_name, model in models.items():
    print(f"Model: {model_name}")
    model.fit(X_train, y_train)
    y_pred = cross_val_predict(model, X_test, y_test, cv=5)
    print(classification_report(y_test, y_pred))
    print("\n")


In [ ]:
#Test ekranı 
# Kullanıcıdan mesaj al ve sınıflandır
message = input("Sınıflandırmak istediğiniz mesajı girin: ")
predictions = classify_message(message, vectorizer, models)
print("Tahminler:")
for model_name, prediction in predictions.items():
    print(f"{model_name}: {prediction}")